<a href="https://colab.research.google.com/github/Alepescinaa/ScientificTools/blob/main/Project1/Cp1/PINN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercises: physics-informed neural network

Exercise on the implementation of physics-informed neural network.

Date: 2024

Course: 056936 - SCIENTIFIC COMPUTING TOOLS FOR ADVANCED MATHEMATICAL MODELLING (PAGANI STEFANO) [2023-24].

Example adapted from this [notebook](https://colab.research.google.com/drive/1qBrbgevkSBqqYc8bOPiaoJG1MBrBrluN?usp=share_link).


Let us consider the problem

\begin{aligned}
  & v_f *\sqrt(\nabla u\cdot D\nabla u) =1  \,, \quad (x,y) \in [-1.5,1.5] \times [-1.5,1.5]\,\\
\end{aligned}

where $\nu$ is unknown. We consider the PINN framework for solving the state/parameter estimation.

In [1]:
# import required libraries

import tensorflow as tf
import numpy as np
import scipy.io
from tensorflow import keras
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
!pip -q install pyDOE
from pyDOE import lhs  # for latin hypercube sampling


  Preparing metadata (setup.py) ... done


In [2]:
# set seed for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

In [3]:
!git clone https://github.com/Alepescinaa/ScientificTools
%cd ScientificTools/Project1/Cp2

Cloning into 'ScientificTools'...
remote: Enumerating objects: 280, done.
remote: Counting objects: 100% (124/124), done.
remote: Compressing objects: 100% (108/108), done.
remote: Total 280 (delta 32), reused 1 (delta 1), pack-reused 156
Receiving objects: 100% (280/280), 59.83 MiB | 8.50 MiB/s, done.
Resolving deltas: 100% (70/70), done.
/content/ScientificTools/Project1/Cp2


In [4]:

# loading of the dataset

CP2data = np.load("CP2data.npz")
CP2data = CP2data['arr_0']

In [6]:
# loading of the estimate

CP2estimate = np.load("CP2estimate.npz")
CP2estimate = CP2estimate['arr_0']

In [7]:

theta_fiber = CP2estimate[ind_disp][0]
a_ratio  = CP2estimate[ind_disp][1]
y0 = CP2estimate[ind_disp][2]

# Solution

In [8]:
X, Y = np.meshgrid(np.linspace(-1.5,1.5,1501), np.linspace(-1.5,1.5,1501))
X_flat = tf.convert_to_tensor(np.hstack((X.flatten()[:,None],Y.flatten()[:,None])),dtype=tf.float64)
x0 = 1.5

theta0 = np.pi/2 - theta_fiber
a = np.array([np.cos(theta0), np.sin(theta0)]).T
b = np.array([np.cos(theta0-np.pi/2), np.sin(theta0-np.pi/2)]).T

D = ( (1/a_ratio)*np.tensordot( a, a,  axes=0) + np.tensordot( b, b,  axes=0)  )
print(D)

D_inv = np.linalg.inv(D)
sigma_11 = D_inv[0][0]
sigma_12 = D_inv[0][1]
sigma_21 = D_inv[1][0]
sigma_22 = D_inv[1][1]

#activation_time = anysotropic_FMM( x0 , y0 , X, Y, sigma_11,sigma_12, sigma_21, sigma_22 )

[[ 0.98915396 -0.08798785]
 [-0.08798785  0.28620407]]


In [ ]:
# PINN loss function
def loss(xcl,ycl,xmeas,ymeas,umeas,param):
    umeas_pred = PINN(tf.concat([xmeas,ymeas],1))
    r_pred   = r_PINN(xcl,ycl,param)

    # loss components
    mse_meas  = tf.reduce_mean(tf.pow(umeas-umeas_pred,2))
    mse_r  = tf.reduce_mean(tf.pow(r_pred,2))

    return mse_r+mse_meas # mse_0+mse_r+mse_lb+mse_ub

# neural network weight gradients
@tf.function
def grad(model,xcl,ycl,xmeas,ymeas,umeas,param):
    with tf.GradientTape(persistent=True) as tape:
        loss_value = loss(xcl,ycl,xmeas,ymeas,umeas,param)
        grads = tape.gradient(loss_value,model.trainable_variables)
        grad_param = tape.gradient(loss_value,param)
    return loss_value, grads, grad_param


In [ ]:
# collocation points
Ncl = 5000
Xcl = lhs(2,Ncl)
xcl = tf.expand_dims(tf.cast(-1.5+(3.0)*Xcl[:,0],dtype=tf.float64),axis=-1)
ycl = tf.expand_dims(tf.cast(-1.5+(3.0)*Xcl[:,1],dtype=tf.float64),axis=-1)
X_coll = tf.concat([xcl,ycl],1)

# measurement points
ind_disp=5
xmeas = CP2data[ind_disp][0]
ymeas = CP2data[ind_disp][1]
tmeas = CP2data[ind_disp][2]
xmeas = tf.constant(xmeas.reshape(20, 1), dtype=tf.float64)
ymeas = tf.constant(ymeas.reshape(20, 1), dtype=tf.float64)
tmeas = tf.constant(tmeas.reshape(20, 1), dtype=tf.float64)




In [ ]:
# training loop

# initialize new instance of NN
PINN = tf.keras.Sequential([
    tf.keras.layers.Dense(8, activation='tanh', input_shape=(2,),kernel_initializer="glorot_normal",dtype=tf.float64),
    tf.keras.layers.Dense(32, activation='tanh',kernel_initializer="glorot_normal",dtype=tf.float64),
    tf.keras.layers.Dense(1,activation=None,kernel_initializer="glorot_normal",dtype=tf.float64)
])



In [ ]:

# residual computation based on AD
@tf.function
def r_PINN(x,y,param):
    u    = PINN(tf.concat([x,y], 1))
    u_x  = tf.gradients(u,x)[0]
    u_y  = tf.gradients(u,y)[0]
    x0 = 1.5
    theta0 = np.pi/2 - param[0]
    a = np.array([np.cos(theta0), np.sin(theta0)]).T
    b = np.array([np.cos(theta0-np.pi/2), np.sin(theta0-np.pi/2)]).T
    D = ( (1/param[])*np.tensordot( a, a,  axes=0) + np.tensordot( b, b,  axes=0)  )
    return  param*(tf.sqrt(sigma_f*u_x**2 + sigma_s*u_y**2) )-1


# Adam optimizer
tf_optimizer = tf.keras.optimizers.Adam(learning_rate=0.003,beta_1=0.99)

# parameter variable
param = tf.Variable(0.1,trainable=True,dtype=tf.float64)
for iter in range(20000):


  # compute gradients using AD
  loss_value,grads,grad_param = grad(PINN,xcl,ycl,xmeas,ymeas,tmeas,param)

  # update neural network weights
  tf_optimizer.apply_gradients(zip(grads+[grad_param],PINN.trainable_variables+[param]))



  # display intermediate results
  if ((iter+1) % 200 == 0):
    print('iter =  '+str(iter+1))
    print('loss = {:.4f}'.format(loss_value))
    print('coefficient estimate = {:.4f}'.format((param.numpy())))
    PINN_flat = PINN(X_flat)
    mse = 0.0
    time_pred = tf.reshape(PINN_flat, (1501, 1501))
    xmeas=xmeas.numpy().squeeze()
    ymeas=ymeas.numpy().squeeze()
    tmeas=tmeas.numpy().squeeze()
    time_pred=time_pred.numpy()
    for k in range(20):
            i, j = np.where((X == xmeas[k]) & (Y == ymeas[k]))
            mse+= (time_pred[i, j] - tmeas[k]) ** 2
    print('mse error: %.4e' % (np.sqrt(mse/20)))
    xmeas = tf.constant(xmeas.reshape(20, 1), dtype=tf.float64)
    ymeas = tf.constant(ymeas.reshape(20, 1), dtype=tf.float64)
    tmeas = tf.constant(tmeas.reshape(20, 1), dtype=tf.float64)




iter =  200
loss = 1.0018
coefficient estimate = 0.5593


<ipython-input-151-db87901cdf8c>:41: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('mse error: %.4e' % (np.sqrt(mse/20)))


mse error: 4.5650e-02
iter =  400
loss = 0.9999
coefficient estimate = 0.8507
mse error: 1.3076e-02
iter =  600
loss = 0.9999
coefficient estimate = 1.0558
mse error: 7.2159e-03
iter =  800
loss = 0.9998
coefficient estimate = 1.2502
mse error: 8.3771e-03
iter =  1000
loss = 0.9997
coefficient estimate = 1.4952
mse error: 9.1105e-03
iter =  1200
loss = 0.9997
coefficient estimate = 1.7926
mse error: 1.1130e-02
iter =  1400
loss = 0.9996
coefficient estimate = 2.1557
mse error: 1.3317e-02
iter =  1600
loss = 0.9994
coefficient estimate = 2.5914
mse error: 1.6110e-02
iter =  1800
loss = 0.9992
coefficient estimate = 3.1043
mse error: 1.9414e-02
iter =  2000
loss = 0.9989
coefficient estimate = 3.6948
mse error: 2.3115e-02
iter =  2200
loss = 0.9983
coefficient estimate = 4.3632
mse error: 2.6843e-02
iter =  2400
loss = 0.9942
coefficient estimate = 5.2196
mse error: 5.6946e-02
iter =  2600
loss = 0.9242
coefficient estimate = 6.4566
mse error: 1.4101e-01
iter =  2800
loss = 0.4281
coeffi

In [ ]:
#Display results

fig = plt.figure(figsize=(16,9),dpi=150)
#fig = plt.figure()
#fig.subplots_adjust(wspace=0.3)
plt.style.use('default')
ax = fig.add_subplot(1,3,1)
ax.set_aspect(1)
im = plt.pcolor(X_plot, Y_plot, u_true)
plt.scatter(xmeas,ymeas,marker='x',s=3)
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(im,cax=cax)
#ax.set_yticklabels(['-1.0','-0.6','-0.2','0.2','0.6','1.0'])
ax.set_title('Exact Solution',fontsize=16)

ax = fig.add_subplot(1,3,2)
ax.set_aspect(1)
im = plt.pcolor(X_plot, Y_plot, np.reshape(PINN_flat,(N_h,N_h)))
plt.scatter(xmeas,ymeas,marker='x',s=3)
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(im,cax=cax)
ax.set_title('PINN Prediction'.format(err),fontsize=16)

ax = fig.add_subplot(1,3,3)
ax.set_aspect(1)
im = plt.pcolor(X_plot, Y_plot, np.abs( np.reshape(PINN_flat,(N_h,N_h)) -u_true ) )
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(im,cax=cax)
ax.set_title('L2 error = {:.4f}'.format(err),fontsize=16)

scale parametri
inizializzazione parametri incogniti ma nontrainable
